# Zad. 

Otagować (czyli wskazać słowa "kluczowe", Latent Dirichlet Allocation) teksty korpusu 20newsgroups.

In [1]:
from gensim import corpora, models, similarities, matutils
import numpy as np
import os 

### Można sczytać dane bez nagłówków i stopek

In [2]:
to_remove= ('headers', 'footers', 'quotes')

from sklearn.datasets import fetch_20newsgroups
newsgroups_train = fetch_20newsgroups(subset='all', shuffle=True, remove = to_remove)

In [3]:
newsgroups_train.data[0]

"\n\nI am sure some bashers of Pens fans are pretty confused about the lack\nof any kind of posts about the recent Pens massacre of the Devils. Actually,\nI am  bit puzzled too and a bit relieved. However, I am going to put an end\nto non-PIttsburghers' relief with a bit of praise for the Pens. Man, they\nare killing those Devils worse than I thought. Jagr just showed you why\nhe is much better than his regular season stats. He is also a lot\nfo fun to watch in the playoffs. Bowman should let JAgr have a lot of\nfun in the next couple of games since the Pens are going to beat the pulp out of Jersey anyway. I was very disappointed not to see the Islanders lose the final\nregular season game.          PENS RULE!!!\n\n"

In [4]:
print(newsgroups_train.data[0])



I am sure some bashers of Pens fans are pretty confused about the lack
of any kind of posts about the recent Pens massacre of the Devils. Actually,
I am  bit puzzled too and a bit relieved. However, I am going to put an end
to non-PIttsburghers' relief with a bit of praise for the Pens. Man, they
are killing those Devils worse than I thought. Jagr just showed you why
he is much better than his regular season stats. He is also a lot
fo fun to watch in the playoffs. Bowman should let JAgr have a lot of
fun in the next couple of games since the Pens are going to beat the pulp out of Jersey anyway. I was very disappointed not to see the Islanders lose the final
regular season game.          PENS RULE!!!




In [5]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [6]:
print(len(newsgroups_train.target_names))

20


In [7]:
print(newsgroups_train.target)

[10  3 17 ...  3  1  7]


In [8]:
data = []
for j in range(len(newsgroups_train.target_names)):
    data.append([newsgroups_train.data[i] for i in range(newsgroups_train.target.shape[0]) if newsgroups_train.target[i] == j])

#biore okreslona ilosc danych z kazdej klasy

categories = 5
num=500
X_set = []
Y_set =[]
for i in range(categories):
    X_set = X_set + data[i][:num]
    Y_set = Y_set + [i]*num

# Zad
Wykonaj przekształcenie Tf-Idf

```python
from sklearn.feature_extraction.text import TfidfVectorizer
```

In [9]:
from nltk.corpus import stopwords
import string
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer

def remove_punctuation(words):
    remove_list =  stopwords.words('english') + list(string.punctuation) + ["'s",'′','--', '."', '!"', '?"', ',"', '``', "''", "\\\\",'⁄']
    return [w for w in words if not w in set(remove_list)]

def remove_nummbers(words):
    return [w for w in words if not w.isdigit()]

def to_lower(words):
    return [w.lower() for w in words]


def my_tokenizer(sent):
    tokens = word_tokenize(sent)
    tokens = to_lower(tokens)    
    tokens = remove_punctuation(tokens)
    tokens = remove_nummbers(tokens)
    stemmer = PorterStemmer()
    res = [stemmer.stem(word) for word in tokens]
    return res 



tfidf_vectorizer = TfidfVectorizer(tokenizer=my_tokenizer, stop_words=stopwords.words('english') + list(string.punctuation))
tfidf_matrix = tfidf_vectorizer.fit_transform(X_set)
# print( tfidf_vectorizer.vocabulary_ )

In [10]:
print("----")
print(tfidf_matrix.todense().shape)
print(tfidf_matrix.todense())

----
(2500, 56894)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


# Zad 
Stwórz model języka w bibliotece **gensim** na podstawie reprezentacji **Tf-Idf**

In [11]:
corpus_sparse = matutils.Sparse2Corpus(tfidf_matrix, documents_columns=False)
dictionary_list = tfidf_vectorizer.get_feature_names()
dictionary_sparse_dict = {i:j for i,j in enumerate(dictionary_list)}
dictionary = corpora.Dictionary.from_corpus(corpus_sparse, id2word=dictionary_sparse_dict)

# Zad 
Stwórz model **Lda** (Latent Dirichlet Allocation) z
* **num_topics=20**

In [12]:
model = models.LdaModel(corpus=corpus_sparse, id2word=dictionary, num_topics=20)#, alpha='auto', eta='auto', iterations=100)

# Wizualizacja modelu LDA:

pyLDAvis

In [18]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

In [19]:
pyLDAvis.gensim.prepare(model, corpus_sparse, dictionary)

C:\Users\przem\AppData\Local\Continuum\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.092639 -0.001032       1        1  50.807369
18     0.020303  0.029459       2        1   6.719706
3      0.021699 -0.025617       3        1   5.787285
15     0.009183  0.000499       4        1   4.647052
17     0.010240 -0.001434       5        1   4.258343
6     -0.001123 -0.000993       6        1   2.766036
11     0.006174  0.000891       7        1   2.676171
19    -0.008282  0.002483       8        1   2.406700
0     -0.003417 -0.001985       9        1   2.114325
12    -0.003710  0.001663      10        1   2.008828
2     -0.010941  0.000703      11        1   1.846149
10    -0.010682  0.000367      12        1   1.825696
9     -0.009995  0.000609      13        1   1.712378
1     -0.012967  0.000116      14        1   1.691221
16    -0.010325 -0.001436      15        1   1.655260
7     -0.016366  0.000211      16        1   1.523331
13    -0.014473 -0.001728      17        1   1.515807
5     -0.016242 -0.000448      18        1   1.492775
14    -0.018772 -0.000249      19        1   1.338161
8     -0.022942 -0.002079      20        1   1.207410, topic_info=      Category       Freq       Term      Total  loglift  logprob
term                                                             
47606  Default  21.000000       simm  21.000000  30.0000  30.0000
36290  Default  18.000000        mac  18.000000  29.0000  29.0000
38185  Default  34.000000    monitor  34.000000  28.0000  28.0000
46055  Default   6.000000        rom   6.000000  27.0000  27.0000
24479  Default  28.000000      drive  28.000000  26.0000  26.0000
21707  Default   4.000000     centri   4.000000  25.0000  25.0000
22809  Default   9.000000        cpu   9.000000  24.0000  24.0000
19056  Default   5.000000       appl   5.000000  23.0000  23.0000
52537  Default  12.000000       vram  12.000000  22.0000  22.0000
21421  Default   5.000000       cabl   5.000000  21.0000  21.0000
44627  Default   5.000000     quadra   5.000000  20.0000  20.0000
49577  Default   6.000000       tech   6.000000  19.0000  19.0000
18179  Default  11.000000    acceler  11.000000  18.0000  18.0000
28988  Default   5.000000       heat   5.000000  17.0000  17.0000
43489  Default   8.000000  processor   8.000000  16.0000  16.0000
22090  Default  18.000000      clock  18.000000  15.0000  15.0000
43434  Default   7.000000      print   7.000000  14.0000  14.0000
48127  Default  12.000000      speed  12.000000  13.0000  13.0000
47791  Default  16.000000       slot  16.000000  12.0000  12.0000
47086  Default  11.000000       scsi  11.000000  11.0000  11.0000
43475  Default  25.000000    problem  25.000000  10.0000  10.0000
51465  Default  25.000000        use  25.000000   9.0000   9.0000
3933   Default  18.000000        ...  18.000000   8.0000   8.0000
39466  Default  28.000000        n't  28.000000   7.0000   7.0000
33203  Default  22.000000       know  22.000000   6.0000   6.0000
37449  Default   4.000000        mhz   4.000000   5.0000   5.0000
54120  Default  22.000000      would  22.000000   4.0000   4.0000
30515  Default   7.000000         ii   7.000000   3.0000   3.0000
36329  Default  12.000000     machin  12.000000   2.0000   2.0000
36347  Default   6.000000  macintosh   6.000000   1.0000   1.0000
...        ...        ...        ...        ...      ...      ...
15094  Topic20   0.011466   =eugene=   0.179077   1.6683  -9.5695
23953  Topic20   0.046384     dialog   0.732194   1.6576  -8.1719
6950   Topic20   0.011330  1360x1024   0.178942   1.6571  -9.5814
21883  Topic20   0.011278     chines   0.178890   1.6528  -9.5861
22889  Topic20   0.013577      cross   0.221663   1.6239  -9.4005
27634  Topic20   0.067123       game   1.632555   1.2253  -7.8024
18297  Topic20   0.091830    address   2.458297   1.1294  -7.4890
21954  Topic20   0.017357       cica   0.305353   1.5492  -9.1549
47609  Topic20   0.048056      simpl   1.256738   1.1528  -8.13